In [1]:
import os
import pandas as pd
import openai
from util import extract_text_from_resume
import json
import re
import textwrap
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import random

from dotenv import load_dotenv, find_dotenv

def wraptext(text, width=100):
    wrapper = textwrap.TextWrapper(width=width)
    word_list = wrapper.wrap(text=text)

    for element in word_list:
        print(element)

# Load environment variables from .env file
_=load_dotenv(find_dotenv())

# Set OpenAI API key
openai.api_key = os.environ['OPENAI_API_KEY']


In [2]:
# List of 100 companies
companies = [
    'Apple Inc.',
    'Google LLC',
    'Microsoft Corporation',
    'Amazon.com Inc.',
    'Facebook Inc.',
    'Berkshire Hathaway Inc.',
    'Johnson & Johnson',
    'Walmart Inc.',
    'Exxon Mobil Corporation',
    'Samsung Electronics Co. Ltd.',
    'The Walt Disney Company',
    'Intel Corporation',
    'Cisco Systems Inc.',
    'Oracle Corporation',
    'The Coca-Cola Company',
    'PepsiCo Inc.',
    'Nike Inc.',
    'Procter & Gamble Co.',
    'McDonald’s Corporation',
    '3M Company',
    'IBM Corporation',
    'Netflix Inc.',
    'Visa Inc.',
    'JPMorgan Chase & Co.',
    'General Electric Company',
    'Adobe Inc.',
    'Tesla Inc.',
    'eBay Inc.',
    'Pfizer Inc.',
    'Boeing Company',
    'NVIDIA Corporation',
    'Twitter Inc.',
    'Starbucks Corporation',
    'General Motors Company',
    'Ford Motor Company',
    'The Home Depot Inc.',
    'BASF SE',
    'Qualcomm Inc.',
    'Abbott Laboratories',
    'FedEx Corporation',
    'AT&T Inc.',
    'Verizon Communications Inc.',
    'DHL International GmbH',
    'Uber Technologies Inc.',
    'Mastercard Incorporated',
    'LVMH Moët Hennessy Louis Vuitton SE',
    'Unilever PLC',
    'Siemens AG',
    'Airbus SE',
    'Costco Wholesale Corporation',
    'HSBC Holdings plc',
    'Vodafone Group Plc',
    'Hitachi Ltd.',
    'AstraZeneca PLC',
    'Bayer AG',
    'Merck & Co., Inc.',
    'Novartis AG',
    'Estée Lauder Companies Inc.',
    'Wells Fargo & Company',
    'SAP SE',
    'Royal Dutch Shell plc',
    'BP plc',
    'TotalEnergies SE',
    'Lockheed Martin Corporation',
    'Rolls-Royce Holdings plc',
    '3D Systems Corporation',
    'United Parcel Service, Inc.',
    'Panasonic Corporation',
    'Accenture plc',
    'Chevron Corporation',
    'Marriott International, Inc.',
    'American Express Company',
    'Salesforce.com, inc.',
    'Volvo Group',
    'Roche Holding AG',
    'GlaxoSmithKline plc',
    'Bristol Myers Squibb Company',
    'Lilly Eli & Co',
    'Toyota Motor Corporation',
    'Volkswagen AG',
    'Honda Motor Co., Ltd.',
    'Hyundai Motor Company',
    'Goldman Sachs Group, Inc.',
    'Bank of America Corporation',
    'PayPal Holdings, Inc.',
    'Spotify Technology S.A.',
    'Square, Inc.',
    'LinkedIn Corporation',
    'Zoom Video Communications, Inc.',
    'SpaceX',
    'Nestlé S.A.',
    'Diageo plc',
    'Prologis, Inc.',
    'Anheuser-Busch InBev SA/NV',
    'The Kraft Heinz Company',
    'Morgan Stanley',
    'Citigroup Inc.',
    'BlackRock, Inc.',
    'Alibaba Group Holding Limited',
    'Tencent Holdings Ltd.'
]
universities = [
    "Harvard University",
    "Stanford University",
    "Massachusetts Institute of Technology",
    "California Institute of Technology",
    "University of Cambridge",
    "University of Oxford",
    "Princeton University",
    "Yale University",
    "Columbia University",
    "University of Chicago",
    "University of California, Berkeley",
    "University of California, Los Angeles",
    "University of California, San Diego",
    "University of California, San Francisco",
    "University of California, Davis",
    "University of California, Irvine",
    "University of California, Santa Barbara",
    "University of California, Santa Cruz",
    "University of Pennsylvania",
    "Cornell University",
    "Brown University",
    "Dartmouth College",
    "Duke University",
    "Johns Hopkins University",
    "Northwestern University",
    "University of Michigan",
    "University of Wisconsin-Madison",
    "University of Washington",
    "University of Texas at Austin",
    "University of Illinois at Urbana-Champaign",
    "University of Toronto",
    "McGill University",
    "University of British Columbia",
    "University of Montreal",
    "University of Melbourne",
    "University of Sydney",
    "University of Queensland",
    "University of New South Wales",
    "University of Auckland",
    "University of Tokyo",
    "Kyoto University",
    "University of Hong Kong",
    "National University of Singapore",
    "Peking University",
    "Tsinghua University",
    "Seoul National University",
    "University of Cape Town",
    "University of São Paulo",
    "University of Buenos Aires",
    "University of Nairobi"
]

degree_types = [
    # Physics
    "Bachelor of Science in Physics",
    "Master of Science in Physics",
    "Ph.D. in Physics",
    "Bachelor of Science in Astrophysics",
    "Master of Science in Astrophysics",
    "Ph.D. in Astrophysics",
    "Bachelor of Science in Theoretical Physics",
    "Master of Science in Theoretical Physics",
    "Ph.D. in Theoretical Physics",
    "Bachelor of Science in Experimental Physics",
    "Master of Science in Experimental Physics",
    "Ph.D. in Experimental Physics",
    "Bachelor of Science in Quantum Physics",
    "Master of Science in Quantum Physics",
    "Ph.D. in Quantum Physics",
    "Bachelor of Science in Condensed Matter Physics",
    "Master of Science in Condensed Matter Physics",
    "Ph.D. in Condensed Matter Physics",
    "Bachelor of Science in Nuclear Physics",
    "Master of Science in Nuclear Physics",
    "Ph.D. in Nuclear Physics",
    
    # Mathematics
    "Bachelor of Science in Mathematics",
    "Master of Science in Mathematics",
    "Ph.D. in Mathematics",
    "Bachelor of Science in Applied Mathematics",
    "Master of Science in Applied Mathematics",
    "Ph.D. in Applied Mathematics",
    "Bachelor of Science in Pure Mathematics",
    "Master of Science in Pure Mathematics",
    "Ph.D. in Pure Mathematics",
    "Bachelor of Science in Computational Mathematics",
    "Master of Science in Computational Mathematics",
    "Ph.D. in Computational Mathematics",
    "Bachelor of Science in Mathematical Physics",
    "Master of Science in Mathematical Physics",
    "Ph.D. in Mathematical Physics",
    "Bachelor of Science in Statistics",
    "Master of Science in Statistics",
    "Ph.D. in Statistics",
    "Bachelor of Science in Actuarial Science",
    "Master of Science in Actuarial Science",
    "Ph.D. in Actuarial Science",
    
    # Computer Science
    "Bachelor of Science in Computer Science",
    "Master of Science in Computer Science",
    "Ph.D. in Computer Science",
    "Bachelor of Science in Software Engineering",
    "Master of Science in Software Engineering",
    "Ph.D. in Software Engineering",
    "Bachelor of Science in Computer Engineering",
    "Master of Science in Computer Engineering",
    "Ph.D. in Computer Engineering",
    "Bachelor of Science in Information Technology",
    "Master of Science in Information Technology",
    "Ph.D. in Information Technology",
    "Bachelor of Science in Cybersecurity",
    "Master of Science in Cybersecurity",
    "Ph.D. in Cybersecurity",
    
    # Machine Learning
    "Bachelor of Science in Machine Learning",
    "Master of Science in Machine Learning",
    "Ph.D. in Machine Learning",
    "Bachelor of Science in Data Science",
    "Master of Science in Data Science",
    "Ph.D. in Data Science",
    "Bachelor of Science in Artificial Intelligence",
    "Master of Science in Artificial Intelligence",
    "Ph.D. in Artificial Intelligence",
    "Bachelor of Science in Deep Learning",
    "Master of Science in Deep Learning",
    "Ph.D. in Deep Learning",
    "Bachelor of Science in Computational Neuroscience",
    "Master of Science in Computational Neuroscience",
    "Ph.D. in Computational Neuroscience"
]





In [3]:
# list of 30 job titles
job_titles = [
    'Data Engineer',
    'Data Analyst',
    'Data Scientist',
    'Software Engineer'
]


In [4]:
de_skills=['airflow', 'alteryx', 'athena', 'aurora', 'aws', 'azure', 'boto3', 'cassandra', 'cloud computing', 'cloudwatch', 'cockroachdb','cypher query language', 'data factory', 'data lake', 'data lake analytics', 'data management', 'data mapping', 'data migration', 'data mining', 'data munging', 'data pipeline design', 'data pipelines', 'data warehousing', 'database design', 'databricks', 'django', 'docker', 'dynamodb', 'ec2', 'elasticsearch', 'emr', 'etl', 'etl design', 'etl frameworks', 'gcp', 'glue', 'google bigquery', 'google bigtable', 'google cloud compute engine', 'google cloud dataflow', 'google cloud functions', 'google cloud kubernetes engine', 'google cloud pub/sub', 'google cloud spanner', 'google cloud storage', 'google storage', 'hadoop', 'hbase', 'hive', 'impala', 'informatica', 'kinesis', 'knime', 'kubernetes', 'lambda', 'mongodb', 'mpi', 'mysql', 'olap', 'oltp', 'oracle', 'pandas', 'pentaho', 'pig', 'pl/sql', 'postgresql', 'power bi', 'pyspark', 'rds', 'redis', 'redshift', 'relational databases', 's3', 'sas', 'sas enterprise guide', 'scalable systems', 'schema design', 'snowflake', 'solr', 'spark', 'sql', 'sql anywhere', 'sql server', 'sql*plus', 'sql/psm', 'sqlite', 'ssis','scala', 'stata', 'sybase', 'talend', 'teradata', 'unix', 'yarn','distributed systems', 'mariadb', 'performance tuning','neo4j']

da_skills=['agile', 'bayesian networks', 'classification', 'clustering', 'collaborative filtering', 'cognos', 'dimensional modeling', 'dimensionality reduction', 'excel', 'google analytics', 'hypothesis testing', 'looker', 'mathematics', 'matplotlib', 'power bi', 'predictive modeling', 'probability', 'r', 'random forest', 'regression analysis', 'scrum', 'spectral clustering', 'spotfire', 'sql', 'statistical analysis', 'tableau', 'text mining', 'tidyverse', 'time series analysis', 'unsupervised learning', 'web scraping', 'weka','kibana', 'splunk']

ds_skills=['bayesian networks', 'classification', 'clustering', 'collaborative filtering', 'convolutional neural networks', 'cuda', 'deep learning', 'dimensionality reduction', 'ensemble modeling', 'feature engineering', 'h2o', 'hypothesis testing', 'keras', 'machine learning','matlab', 'machine vision', 'mapreduce', 'mathematics', 'matplotlib', 'natural language processing', 'neural networks', 'nlp', 'nltk', 'numpy', 'opencv', 'pandas', 'predictive modeling', 'probability', 'random forest', 'regression analysis', 'recommender systems', 'scikit-learn', 'singular value decomposition','statistical analysis', 'support vector machines', 'tensorflow', 'text mining', 'theano', 'time series analysis', 'torch', 'unsupervised learning', 'Large Language Models', 'natural language processing', 'nlp', 'nltk', 'pytorch','logistic regression']

se_skills=['app development', 'agile', 'android app development', 'angularjs', 'ariflow', 'aws', 'azure', 'clojure', 'cloud computing', 'cloudwatch', 'cuda', 'd3js', 'dask', 'devops', 'django', 'docker', 'electron', 'erlang', 'expressjs', 'fastapi', 'fiori/ui5', 'flask', 'git', 'go', 'gradle', 'graphql', 'grunt', 'gulp', 'haskell', 'hibernate', 'html/css', 'java', 'javafx', 'javascript', 'jenkins', 'jquery', 'junit', 'jupyter notebook', 'kotlin', 'kubernetes', 'lambda', 'lamp stack', 'laravel', 'linux', 'microservices', 'mockito', 'nodejs', 'nosql', 'object oriented abap', 'object-oriented programming', 'objective-c', 'opencv', 'opengl', 'openmp', 'perl', 'php', 'posix threads', 'python', 'qt', 'r', 'rabbitmq', 'rdbms', 'reactjs', 'rest apis', 'ruby', 'ruby on rails', 'rust', 'selenium', 'shell scripting', 'spring', 'spring framework', 'sql', 'sql server', 'swift', 'symfony', 't-sql', 'tableau', 'transact-sql', 't-sql', 'typescript', 'unity', 'unix', 'vuejs', 'web scraping', 'web services', 'webpack', 'xml','distributed systems', 'maven', 'performance tuning', 'pygame', 'pyqt', 'scala', 'tkinter', 'stl']

job_skills=list(set(de_skills+da_skills+ds_skills+se_skills))
print("Number of hard skills for Data Engineer is {}, Data Analyst is {}, Data Scientist is {}, Software Engineer is {}".format(len(de_skills),len(da_skills),len(ds_skills),len(se_skills)))
print("Number of total hard skills are {}".format(len(job_skills)))

Number of hard skills for Data Engineer is 98, Data Analyst is 34, Data Scientist is 47, Software Engineer is 95
Number of total hard skills are 231


In [5]:
data_scientist_titles = ["Data Science",
    "Data Scientist","DataScientist",
    "Machine Learning Engineer",
    "AI Engineer",
    "AI Researcher",
    "Machine Learning Researcher",
    "Computer Vision Engineer",
    "Natural Language Processing Engineer",
    "Deep Learning Engineer",
    "Data Science Specialist",
    "Statistical Modeler",
    "Applied Scientist",
    "Research Scientist",
    "Quantitative Analyst",
    "AI Scientist",
    "Algorithm Developer",
    "Machine Learning Scientist",
    "AI Developer",
    "Data Science Engineer",
    "Machine Learning Specialist",
    "Data Science Analyst",
    "Artificial Intelligence Engineer",
    "Deep Learning Specialist",
    "Predictive Modeler",
    "Quantitative Researcher",
    "AI Consultant",
    "Reinforcement Learning Engineer",
    "Data Science Consultant",
    "Machine Learning Analyst",
    "AI Analyst",
    "Data Science Researcher",
    "ML Engineer","Machine Learning","ML Ops Engineer",
    "ML Researcher",
    "Data Mining Engineer",
    "Data Science Developer",
    "AI Specialist",
    "Algorithm Engineer",
    "Deep Learning Researcher",
    "Machine Intelligence Engineer",
    "Data Analysis Scientist",
    "Advanced Analytics Engineer"
]
data_engineer_titles = ["Cloud Engineer","Engineer, Data","ETL Developer","Data Ops Lead","Data Schema Specialist",
    "Data Engineer","Data Developer",
    "Big Data Engineer",
    "Data Integration Engineer",
    "Data Pipeline Engineer",
    "Data Platform Engineer",
    "ETL Engineer",
    "Big Data Developer",
    "Data Infrastructure Engineer",
    "Data Warehouse Engineer",
    "Streaming Data Engineer",
    "Data Engineering Manager",
    "Database Engineer",
    "Cloud Data Engineer",
    "Azure Data Engineer",
    "AWS Data Engineer",
    "GCP Data Engineer",
    "Hadoop Engineer",
    "Data Architect",
    "Big Data Architect",
    "Data Engineering Consultant",
    "Data Operations Engineer",
    "Data Systems Engineer",
    "Data Analytics Engineer",
    "Real-time Data Engineer",
    "Spark Engineer",
    "NoSQL Engineer",
    "Data Lake Engineer",
    "Data Management Engineer",
    "Data Storage Engineer",
    "Distributed Data Engineer",
    "Data Processing Engineer",
    "Big Data Solutions Engineer",
    "Data Strategy Engineer",
    "Data Science Platform Engineer",
    "Data Engineering Analyst",
    "Data Modeler",
    "Data Integration Architect",
    "Data Engineering Specialist",
    "Data Migration Engineer",
    "DevOps Engineer",
    "Data Engineering Lead"
]

data_analyst_titles = ["Data Analytics","Data Management Analyst","Analytic","Analyst",
    "Data Analysis",
    "Data Analyst",
    "Business Data Analyst",
    "Data Analytics Specialist",
    "Marketing Data Analyst",
    "Financial Data Analyst",
    "Operations Data Analyst",
    "Quantitative Data Analyst",
    "Healthcare Data Analyst",
    "Data Reporting Analyst",
    "Sales Data Analyst",
    "Customer Data Analyst",
    "Data Quality Analyst",
    "Data Visualization Analyst",
    "HR Data Analyst",
    "Supply Chain Data Analyst",
    "Risk Data Analyst",
    "Data Insights Analyst",
    "Data Analyst Intern",
    "Data Governance Analyst",
    "Data Analyst Consultant",
    "SQL Data Analyst",
    "Data Analyst Manager",
    "Data Research Analyst",
    "Data Analyst Lead",
    "Junior Data Analyst",
    "Senior Data Analyst",
    "Data Analyst Associate",
    "Data Analyst Coordinator",
    "Database Analyst",
    "Data Audit Analyst",
    "Data Intelligence Analyst",
    "Fraud Data Analyst",
    "Data Integration Analyst",
    "Data Systems Analyst",
    "Data Analyst I",
    "Data Analyst II",
    "Entry-Level Data Analyst",
    "Strategic Data Analyst"
]

software_engineer_titles = ["Software","Frontend","Backend","Python Engineer","Back End Developer",".Net Developer",
    "Software Engineer","Senior Fullstack Developer",".NET/Angular Engineer",
    "Software Developer","MAC Engineer with Xcode/Swift","Android Engineer","Back End Engineer",
    "Web Developer",
    "Full Stack Developer",
    "Backend Developer",
    "Frontend Developer",
    "Mobile Developer",
    "iOS Developer",
    "Android Developer",
    "Embedded Software Engineer",
    "Software Engineer Intern",
    "Junior Software Engineer",
    "Senior Software Engineer",
    "Software Architect",
    "Application Developer",
    "Java Developer","Java Engineer",
    "Python Developer",
    "C++ Developer",
    "C# Developer",
    "JavaScript Developer",
    "Ruby Developer",
    "PHP Developer",
    "Golang Developer",
    "Scala Developer",
    "Swift Developer",
    "Kotlin Developer",
    "Rust Developer",
    "Software Development Engineer",
    "Software Engineering Manager",
    "Software Test Engineer",
    "Game Developer",
    "UI Developer",
    "UX Developer",
    "Frontend Engineer",
    "Backend Engineer",
    "Full Stack Engineer",
    "Cloud Developer",
    "Systems Software Engineer",
    "Software Development Analyst"
]


job_categories = {
        'data engineer': data_engineer_titles,
        'data scientist': data_scientist_titles,
        'data analyst': data_analyst_titles,
        'software engineer': software_engineer_titles
    }


names = ["Albert Einstein", "Leonardo da Vinci", "Nelson Mandela", "Marie Curie", "Mahatma Gandhi", "Charles Darwin", "William Shakespeare", "Isaac Newton", "Wolfgang Amadeus Mozart", "Abraham Lincoln", "Amelia Earhart", "Pablo Picasso", "Helen Keller", "Galileo Galilei", "Jane Austen", "Martin Luther King Jr.", "Steve Jobs", "Thomas Edison", "Rosa Parks", "Winston Churchill", "Vincent van Gogh", "Mother Teresa", "Frida Kahlo", "Ada Lovelace", "Stephen Hawking", "Rosalind Franklin", "Walt Disney", "Coco Chanel", "Neil Armstrong", "Bill Gates", "Elvis Presley", "Queen Elizabeth II", "Muhammad Ali", "Oprah Winfrey", "J.K. Rowling", "Michael Jordan", "Malala Yousafzai", "Marie Antoinette", "Dalai Lama", "Aristotle", "Cleopatra", "Alexander the Great", "Cristiano Ronaldo", "Alberto Santos-Dumont", "Fidel Castro", "Sigmund Freud", "Emily Dickinson", "Pele", "Marilyn Monroe",  "John F. Kennedy", "Salvador Dali", "Pierre Curie", "Catherine the Great", "Amadeo Avogadro", "Fyodor Dostoevsky", "George Washington", "Anne Frank", "Mao Zedong", "Mikhail Gorbachev", "Margaret Thatcher", "Leon Trotsky", "Nikola Tesla", "Frederick Douglass", "Eva Perón", "George Orwell", "John Lennon", "Henry Ford", "Johannes Gutenberg", "Louis Pasteur", "Barack Obama", "Angelina Jolie", "Princess Diana", "Bob Dylan", "Andy Warhol", "David Bowie", "Freddie Mercury", "Audrey Hepburn", "Kurt Cobain", "Julius Caesar", "Plato", "Pope Francis", "Leo Tolstoy", "Vladimir Lenin", "Marlon Brando", "Jim Morrison", "John D. Rockefeller", "Rihanna", "Roger Federer", "Diego Maradona", "Che Guevara", "Martha Graham", "Joseph Stalin", "Elizabeth Taylor", "Pierre-Auguste Renoir", "Emily Brontë", "Virginia Woolf", "Lewis Carroll", "John Steinbeck", "Charles Dickens", "Agatha Christie"]




In [6]:
import os
import random
from datetime import datetime, timedelta

# Function to generate a random date in the past
def random_date(start, end):
    return start + timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())))


start_date = datetime(1990, 1, 1)
end_date = datetime.today()

def generate_resumes(names, degree_types, universities, companies, num_resumes=100):
    resumes = []
    for i in range(num_resumes):
        name = random.choice(names)
        degree = random.choice(degree_types)
        university = random.choice(universities)
        # Select random companies
        companies_selected = random.sample(companies, 6)  
        
        # Select a random title and multiple matching skills for each job category
        job_title_ds, skills_ds = random.choice(data_scientist_titles), random.sample(ds_skills, k=5)
        job_title_de, skills_de = random.choice(data_engineer_titles), random.sample(de_skills, k=5)
        job_title_da, skills_da = random.choice(data_analyst_titles), random.sample(da_skills, k=5)
        job_title_se, skills_se = random.choice(software_engineer_titles), random.sample(se_skills, k=5)
        
        # Select random dates for each position
        # Select random dates for each position
        start_dates = [random_date(start_date, end_date) for _ in range(6)]
        end_dates = [start_date + timedelta(days=random.randint(180, 1800)) for start_date in start_dates]  # 180 to 1800 days is roughly 6 months to 5 years

        # Convert dates to strings
        start_dates = [date.strftime('%Y-%m') for date in start_dates]
        end_dates = [date.strftime('%Y-%m') for date in end_dates]
        
        # Generate all possible experiences
        all_experiences = [
            f"- {job_title_ds} at {companies_selected[0]} from {start_dates[0]} to {end_dates[0]} where they acquired the following skills: {', '.join(skills_ds)}",
            f"- {job_title_de} at {companies_selected[1]} from {start_dates[1]} to {end_dates[1]} where they acquired a different set of skills including: {', '.join(skills_de)}",
            f"- {job_title_da} at {companies_selected[2]} from {start_dates[2]} to {end_dates[2]} where they were able to further enhance their skills, specifically: {', '.join(skills_da)}",
            f"- {job_title_se} at {companies_selected[3]} from {start_dates[3]} to {end_dates[3]} where they perfected their craft and demonstrated their adaptability. Key skills acquired include: {', '.join(skills_se)}",
            f"- {job_title_ds} at {companies_selected[4]} from {start_dates[4]} to {end_dates[4]} where they acquired the following skills: {', '.join(skills_ds)}",
            f"- {job_title_de} at {companies_selected[5]} from {start_dates[5]} to {end_dates[5]} where they acquired a different set of skills including: {', '.join(skills_de)}",
        ]
        
        # Select a random number of experiences between 4 and 6
        num_experiences = random.randint(4, 6)
        experiences = random.sample(all_experiences, num_experiences)
        experiences_str = "\n".join(experiences)
        
        # Construct the prompt for the resume
        prompt = f"""
        {name} is a professional with a diverse background. They hold a {degree} from {university} and have held the following positions:

        {experiences_str}

        Please generate a one-page resume for this individual. 
        Do not add any other comment in response.
        """
        messages = [{"role": "user", "content": prompt}]
        response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=messages,
          temperature=1,
          max_tokens=3000
        )
        resume=response.choices[0].message["content"].strip()
        resumes.append(resume)
        # Specify the directory where the resume files will be saved

        
        directory = "/Users/yunjaewon/JohnResume/resume-rater/resume_data/"
        # Save the resume to a .txt file in the specified directory
        filename = os.path.join(directory, f'{name.replace(" ", "_")}_resume.txt')  # Replace spaces with underscores for the filename
        with open(filename, 'w') as file:
            file.write(resume)
       
            
    return resumes


In [7]:
response=generate_resumes(names, degree_types, universities, companies, num_resumes=100)